In [1]:
import pandas as pd
# Defining of the dataframe
# built-in imports
import re
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup, SoupStrainer
from urllib.request import urlopen
import requests
df = pd.DataFrame(columns=['postcode'])
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
table = BeautifulSoup(requests.get('https://www.worldpostalcodes.org/l1/en/au/australia/list/r1/list-of-postcodes-in-victoria',headers = headers).text, "html.parser")
# Collecting Ddata
for row in table.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        postcode = columns[0].text.strip()

        df = df.append({'postcode':postcode}, ignore_index=True)    
postcodes = df['postcode'].tolist()
postcodes = [int(x) for x in postcodes if x]

/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_9736/2250746184.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcode':postcode}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_9736/2250746184.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcode':postcode}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_9736/2250746184.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'postcode':postcode}, ignore_index=True)
/var/folders/7q/psdflw4d7cdc_6hv6hqsr59m0000gn/T/ipykernel_9736/2250746184.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future

In [2]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.
This is by no means production code.
"""
# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 31)  # update this to your liking
postcode_list = postcodes # All the postcode in the Victoria

# begin code
url_links = []
property_metadata = defaultdict(dict)
# generate list of urls to visit
for postcode in postcode_list:
    postcode_url = BASE_URL + f"/rent/?excludedeposittaken=1&sort=default-desc&postcode={postcode}&page="
    for page in N_PAGES:
        url = postcode_url + f"{page}"
        # find the unordered list (ul) elements which are the results, then
        # find all href (a) tags that are from the base_url website.
        bs_object = BeautifulSoup(requests.get(url,headers = headers).text, "html.parser")
        if bs_object.find("div", {"class": "css-18vn4hf"}) is not None:
        # if this page reachs the limit, then break
            break
        else:
            if bs_object.find("ul",{"data-testid": "results"}) is not None:
                # if this page contains the property url then record the url
                index_links = bs_object \
                    .find(
                        "ul",
                        {"data-testid": "results"}
                    ) \
                    .findAll(
                        "a",
                        href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
                    )
                for link in index_links:
                    # if its a property address, add it to the list
                    if 'address' in link['class']:
                        url_links.append(link['href'])

In [3]:
# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    bs_object = BeautifulSoup(requests.get(property_url,headers = headers).text, "html.parser")
    
    if bs_object.find("h1", {"class": "css-164r41r"}) is not None:
        # looks for the header class to get property name
        property_metadata[property_url]['name'] = bs_object \
            .find("h1", {"class": "css-164r41r"}) \
            .text

    
        # looks for the div containing a summary title for cost
        property_metadata[property_url]['cost_text'] = bs_object \
            .find("div", {"data-testid": "listing-details__summary-title"}) \
            .text
    
        property_metadata[property_url]['property_type'] = bs_object \
                .find("div", {"data-testid": "listing-summary-property-type"}) \
                .text
        '''
        property_metadata[property_url]['agent'] = [
        feature.text for feature in bs_object \
            .find("div", {"data-testid": "listing-details__residential-header-agent-copy"}) \
            .findAll("a")]
        '''
        for li_tag in bs_object.find_all('ul', {'data-testid':'listing-summary-strip'}):
            value=[]
            for span_tag in li_tag.find_all('li'):           
                value.append(span_tag.find('strong').text)
                property_metadata[property_url]['property_feature'] = value
    
    
        # extract coordinates from the hyperlink provided
        # i'll let you figure out what this does :P
        property_metadata[property_url]['coordinates'] = [
            float(coord) for coord in re.findall(
                r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
                bs_object \
                    .find(
                        "a",
                        {"target": "_blank", 'rel': "noopener noreferer"}
                    ) \
                    .attrs['href']
            )[0].split(',')
        ]
        property_metadata[property_url]['rooms'] = [
            re.findall(r'\d\s[A-Za-z]+', feature.text) for feature in bs_object \
                .find("div", {"data-testid": "property-features"}) \
                .findAll("span", {"data-testid": "property-features-text-container"})
        ]
    
    
        property_metadata[property_url]['desc'] = re \
            .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
            .strip('</p>')
    

# output to example json in data/raw/

with open('unfiltered.json', 'w') as f:
    dump(property_metadata, f)

KeyboardInterrupt: 

In [5]:
import pandas as pd
df_raw = pd.read_json('../data/raw/2022_house_data.json')
df_raw = df_raw.T
df_raw = df_raw.reset_index(drop=True)
df_raw
#因为原本数据里面没有SA2的code所以要用看看每个SA2区域里面包含多少个邮编

,name,cost_text,property_type,property_feature,coordinates,rooms,desc
0,101/25-33 Wills Street Melbourne VIC 3000,$400,Apartment / Unit / Flat,"[Friday, 7th October 2022, $1738]","[-37.8107757, 144.9569269]","[[1 Bed], [1 Bath], []]",Mary Khudhur
1,2905B/11 Rose Lane Melbourne VIC 3000,$575 per week,Apartment / Unit / Flat,"[Tuesday, 20th September 2022, $2499]","[-37.815446, 144.9541165]","[[2 Beds], [1 Bath], [1 Parking]]",Mikaele Horsnell
2,1901/368 St Kilda Road Melbourne VIC 3000,$1800 Per Week,Apartment / Unit / Flat,"[Available Now, $10800]","[-37.8318086, 144.9710272]","[[3 Beds], [2 Baths], [2 Parking]]",Darren McMullin
3,1211/200 Spencer Street Melbourne VIC 3000,$480 per week,Apartment / Unit / Flat,"[Tuesday, 18th October 2022, $2085]","[-37.816228, 144.9532465]","[[1 Bed], [1 Bath], [1 Parking]]",South Melbourne Leasing
4,1008/380 Little Lonsdale Street Melbourne VIC ...,$400 pw,Apartment / Unit / Flat,"[Friday, 9th September 2022, $1738]","[-37.8121026, 144.9581291]","[[1 Bed], [1 Bath], []]",Shaun Broadbent
...,...,...,...,...,...,...,...
15087,28 Beachcomber Drive Inverloch VIC 3996,$550 wk,House,"[Available Now, $2390]","[-38.6412502, 145.7031282]","[[4 Beds], [2 Baths], []]","class=""css-dxogle"">* Unverified feature<svg a..."
15088,14 Inverloch Parade Inverloch VIC 3996,$420 wk,House,"[Available Now, $1820]","[-38.6287143, 145.7358082]","[[2 Beds], [1 Bath], []]","class=""css-dxogle"">* Unverified feature<svg a..."
15089,10 Hopetoun Street Inverloch VIC 3996,$370 wk,House,"[Available Now, $1610]","[-38.6341243, 145.7306862]","[[2 Beds], [1 Bath], [1 Parking]]","class=""css-dxogle"">* Unverified feature<svg a..."
15090,Inverloch VIC 3996,$475 wk,House,"[Available Now, $2065]","[-38.6314613, 145.7293638]","[[3 Beds], [2 Baths], [1 Parking]]","class=""css-dxogle"">* Unverified feature<svg a..."
